In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which is incompatible.


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the MarianMT model and tokenizer for English to Hindi
model_name = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [ ]:
import pandas as pd
from datasets import Dataset

# Load your dataset
data = pd.read_csv('English_Hindi_Idioms_500.csv')

# Display the first few rows of the dataset along with the column names
print(data.head())
print("\nColumn names:", data.columns)


# Preprocess the dataset
# Define the preprocessing function
def preprocess_function(data):
    inputs = tokenizer(data['English Idiom'].tolist(), max_length=50, truncation=True, padding='max_length')
    targets = tokenizer(data['Hindi Equivalent Idiom'].tolist(), max_length=50, truncation=True, padding='max_length')
    inputs['labels'] = targets['input_ids']
    return inputs


# Prepare dataset for fine-tuning
train_data = preprocess_function(data)
train_dataset = Dataset.from_dict(train_data)


            English Idiom     Hindi Equivalent Idiom
0  A blessing in disguise        Gud mein keeda hona
1          A dime a dozen                Sasta sauda
2    Beat around the bush  Idhar-udhar ki baat karna
3         Bite the bullet         Kadvi goli nigalna
4           Break the ice                 Barf todna

Column names: Index(['English Idiom', 'Hindi Equivalent Idiom'], dtype='object')


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='no',  # Disable evaluation
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()


Step,Training Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


TrainOutput(global_step=93, training_loss=1.258179941485005, metrics={'train_runtime': 805.4094, 'train_samples_per_second': 1.825, 'train_steps_per_second': 0.115, 'total_flos': 19465076736000.0, 'train_loss': 1.258179941485005, 'epoch': 3.0})

In [ ]:
import torch

def translate_idiom(text):
    # Check if a GPU is available and set the device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Move the model to the appropriate device
    model.to(device)

    # Prepare inputs and move them to the same device as the model
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generate the translation
    translated = model.generate(**inputs)

    # Decode the output and return the translation
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Example test after fine-tuning
print(translate_idiom("burn the midnight oil"))


Raat bhar mehnat karna
